Ссылка на ноутбук: https://colab.research.google.com/drive/1rQvdUA_wqLIcSb_IPFGNtPauy0yG9XSz?usp=sharing

# Classes  
Сегодня в меню:
  * cоздание классов, аттрибуты и методы
  * inheritance
  * name mangling
  * staticmethod, classmethod
  * callable objects
  * basic magic methods
  * contexts

### cоздание классов, аттрибуты и методы

In [ ]:
class Class:
    pass

instance = Class()
instance.some_str = "instance str"  # динамическое добавление поля объекту

print(instance.some_str)
print(isinstance(instance, Class))  # проверка принадлежности объекта классу

In [ ]:
another_instance = Class()
print(another_instance.some_str)

In [ ]:
help(Class)

__Q:__ Это не очень хорошо, где конструктор (как в С++), где методы, где ссылка на текущий объект?  
__A:__ **Они есть!**

In [ ]:
class Class:
    
    def __init__(self, some_str_: str = "I'm instance's field"):  # особый "магический метод" aka конструктор
        self.some_str = some_str_

instance = Class()
print(instance.some_str)
another_instance = Class("I'm another instance's field")
print(another_instance.some_str)
print(instance.some_str)

In [ ]:
class Class:
    
    static_some_str = "I'm static field"
    
    def __init__(self, some_str_: str = "I'm instance's field"):
        self.some_str = some_str_
        
    def print_some_str(self):
        print(f"Class instance field `some_str`='{self.some_str}'")

instance = Class("I'm another instance's field")
instance.print_some_str()

__Q:__ А а что значит self в определении метода?

__A:__ Когда мы вызываем метод как ```<objname>.<methodname>(<arg1>, ..., <argN>)```, нулевым аргументом передается ссылка на ```obj``` (в качестве ```self```)

In [ ]:
class MyLittleClass:
    
    def method_without_self(arg):
        print(arg)
        
    def method_with_self(self, arg):
        print(arg)

In [ ]:
obj = MyLittleClass()
obj.method_with_self('i am an argument')
obj.method_without_self('i am another argument') # здесь мы на самом деле передаем по два аргумента, self и arg

__Q:__ А как же тогда их вызывать?!

__A:__ Они не привязаны к инстансу (потому что не имеют доступа к его локальным данным), зато привязаны к классу

In [ ]:
MyLittleClass.method_without_self('i am another argument')  # а здесь мы передаем только один аргумент

__Q:__ Можно ли "оторвать" метод от инстанса?

__A:__ Ну, попробуем

In [ ]:
func = MyLittleClass.method_without_self
func("hello")

In [ ]:
func2 = MyLittleClass.method_with_self
func2("hello")  # передаем один аргумент

In [ ]:
func3 = obj.method_with_self
func3("hello")  # передаем два аргумента

In [ ]:
func2(MyLittleClass(), "hello")  # ой, нам же ещё нужен объект для self!

__Q:__ А наоборот?

__A:__ Да это же питон. Конечно, можно!

In [ ]:
obj.get_color()

In [ ]:
def get_color_function(self):
    return self.color

MyLittleClass.get_color = get_color_function
obj = MyLittleClass()
obj.get_color()

Ах да, цвета-то у нас нет. Но не беда, это же питон!

In [ ]:
obj.color = 'pink'
obj.get_color()

__Q:__ А как же узнать, что мы уже определили, а что нет?

__A:__ Легко!

In [ ]:
print(dir(obj))

In [ ]:
# оставим только методы
print([name for name in dir(obj) if callable(getattr(obj, name))])

### inheritance

In [ ]:
class Animal:

    some_value = "animal"

    def __init__(self):
        print("i am an animal")
    
    def speak(self):
        raise NotImplementedError('i don\'t know how to speak')

        
class Cat(Animal):
    
    some_value = "cat"
    
    def __init__(self):
        super().__init__()
        print("i am a cat")
    
    def speak(self):
        print('meoooow')

        
class Hedgehog(Animal):
    
    def __init__(self):
        super().__init__()
        print("i am a hedgehog")

        
class Dog(Animal):
    
    some_value = "dog"
    
    def __init__(self):
        super().__init__()
        print("i am a dog")

        
class CatDog(Cat, Dog):  # ромбовидное наследование возможно, но не делайте так, пожалуйста!
    
    def __init__(self):
        super().__init__()
        print("i am a CatDog!")

In [ ]:
animal = Animal()
animal.some_value

In [ ]:
cat = Cat()
cat.some_value  # переопределено

In [ ]:
hedgehog = Hedgehog()
hedgehog.some_value  # не переопределено

In [ ]:
dog = Dog()
dog.some_value  # переопределено

In [ ]:
catdog = CatDog()
catdog.some_value

__Q:__ А как определяется порядок?
    
__A:__ Порядок перечисления родителей важен!

In [ ]:
class CatDog(Dog, Cat):  # теперь наоборот, найдите два отличия!
    def __init__(self):
        super().__init__()
        print("i am a CatDog!")

catdog = CatDog()
catdog.some_value

##### << Пояснительная бригада: начало>>

In [ ]:
class A(object):

    def print(self):
        print("Class А")
    pass


class B(A):
    
    def print(self):
        super().print()
        print("Class B")

    pass


class C(A):

    def print(self):
        super().print()
        print("Class C")
    
    pass


class D(B, C):

    pass

In [ ]:
d = D()
d.print()

In [ ]:
D.mro()

In [ ]:
class E(C, B):

    pass

class F(D, E):

    pass

##### << Пояснительная бригада: конец>>

__Q:__ А что с методами?
    
__A:__ Всё то же, что и с атрибутами!

### name mangling

In [ ]:
class VeryPrivateDataHolder:
    def __init__(self):
        self._secret = 1
        self.__very_secret = 2

In [ ]:
obj = VeryPrivateDataHolder()
print(obj._secret)
print(obj.__very_secret)

__Q:__ То есть, в питоне всё-таки есть приватность?

__A:__ Ну...

Любой атрибут внутри определения класса ```classname``` вида ```__{attr}``` (```attr``` при этом имеет не более одного ```_``` в конце) подменяется на ```_{classname}__{attr}```. Таким образом, внутри классов можно иметь "скрытые" приватные атрибуты, которые не "видны" экземплярам класса и классам-наследникам.

In [ ]:
obj._VeryPrivateDataHolder__very_secret  # а так вообще никогда не делайте, особенно с чужими классами

In [ ]:
obj._VeryPrivateDataHolder__very_secret = 'new secret'
obj._VeryPrivateDataHolder__very_secret

In [ ]:
class DerivedVeryPrivateDataHolder(VeryPrivateDataHolder):
    def __init__(self):
        super().__init__()

obj = DerivedVeryPrivateDataHolder()

In [ ]:
print(obj._secret)
print(obj.__very_secret)

In [ ]:
print(obj._VeryPrivateDataHolder__very_secret)
print(obj._DerivedVeryPrivateDataHolder__very_secret)

### staticmethod, classmethod

`@staticmethod` — используется для создания метода, который ничего не знает о классе или экземпляре, через который он был вызван. Он просто получает переданные аргументы, без неявного первого аргумента, и его определение неизменяемо через наследование.

Проще говоря, `@staticmethod` — это вроде обычной функции, определенной внутри класса, которая не имеет доступа к экземпляру, поэтому ее можно вызывать без создания экземпляра класса.

---

`@classmethod` — это метод, который получает класс в качестве неявного первого аргумента, точно так же, как обычный метод экземпляра получает экземпляр. Это означает, что вы можете использовать класс и его свойства внутри этого метода, а не конкретного экземпляра.

Проще говоря, `@classmethod` — это обычный метод класса, имеющий доступ ко всем атрибутам класса, через который он был вызван. Следовательно, `@classmethod` — это метод, который привязан к классу, а не к экземпляру класса.

In [ ]:
class MyClass:
    
    clsval = 0
    
    def __init__(self,val):
        self.objval = val

    def set_val(self,val):
        type(self).clsval = val  # атрибут класса 
        self.objval = val        # атрибут объекта 
    
    @staticmethod  # можно вызывать и как obj.static_set_val(val) и как MyClass.static_set_val(val)!
    def static_set_val(val):
        MyClass.clsval = val
        
    @classmethod  # передаёт класс первым аргументом
    def class_set_val(cls, val):
        cls.clsval = val

In [ ]:
obj = MyClass(5)
print('clsval', obj.clsval, 'objval', obj.objval)

obj.set_val(9)
print('clsval', obj.clsval, 'objval', obj.objval)

obj.static_set_val(4)
print('clsval', obj.clsval, 'objval', obj.objval)

MyClass.static_set_val(3)
print('clsval', obj.clsval, 'objval', obj.objval)

MyClass.class_set_val(7)
print('clsval', obj.clsval, 'objval', obj.objval)

### callable objects

In [ ]:
class Adder:

    def __init__(self, x):
        self.x = x

    def __call__(self, y):
        return self.x + y
    
adder = Adder(10)

adder(14)

### basic magic methods

In [ ]:
import random

class Vector:        
    def __init__(self, x=0, y=0, color=None):
        print("initializing a vector")
        if type(x) != int or type(y) != int:
            raise AttributeError('x and y should be int')
        
        self._x = x
        self._y = y
        self._color = color
    
    def get_x(self):
        return self._x
    
    def get_y(self):
        return self._y

Методы с двойным подчеркиванием в начале и конце имени имеют особое значение. 

Мы уже знакомы с `__init__` и `__call__`, пора узнать и об остальных

In [ ]:
vector = Vector(1, 2, 'red')
str(vector)

In [ ]:
class VectorWithStr(Vector):
    def __str__(self):
        return 'vector ({}, {}) of color {}'.format(self._x, self._y, self._color)

In [ ]:
vector = VectorWithStr(1, 2, 'red')
str(vector)

__Q:__ Преобразование в строку? Это всё?

__A:__ Конечно, нет. Неявные преобразования иногда происходят там, где мы их не ожидаем

In [ ]:
print(vector)

In [ ]:
mylist = [vector]
print(mylist)

__Q:__ А откуда опять "некрасивые" строки?!  

__A:__ В питоне используется два способа приведения к строке. Это функции `str` и `repr`, которые отличаются своим назначением. 

`str` используется там, где нужна человекочитаемость, а `repr` реализуется так, чтобы можно было однозначно определить, о каком объекте идет речь. Если `repr` не реализован, используется стандартный вариант, а если не реализован `str`, то вместо него используется `repr`. 

Попробуем?

In [ ]:
class VectorWithRepr(Vector):
    def __repr__(self):
        return 'vector representation (x: {}, y: {}, color: {})'.format(self._x, self._y, self._color)

In [ ]:
vector = VectorWithRepr(1, 2, 'red')

print(vector)
mylist = [vector]
print(mylist)

In [ ]:
class VectorWithBothReprAndStr(VectorWithRepr, VectorWithStr):
    pass

In [ ]:
vector = VectorWithBothReprAndStr(1, 2, 'red')
# вот здесь должны получиться разные значения
print(vector)
print([vector])

### contexts

Контекстом (он же менеджер) следит за инициализацией и финализацией данного контекста, в частности определяет действия которые должны происходить до и после выполнения блока кода.

Обычно используется совместно с инструкцией `with`, позволяющей выполнение описанного в ней блока в некотором контексте исполнения. Однако не возбраняется и прямое обращение к методам менеджера.

Менеджеры контекста часто используются в задачах сходных с:
 * сохранением и восстановлением глобального состояния;
 * блокировкой и разблокировкой ресурсов;
 * открытием и закрытиям файлов.

В сущности, признаком принадлежности к типу является следование протоколу контекстного менеджера , характеризующемуся наличием реализаций у пользовательского типа двух методов:

 * `__enter__()`;
 * `__exit__()`.

In [ ]:
class VectorWithContextManager(VectorWithBothReprAndStr):
    def __enter__(self):
        print('entering context')
    def __exit__(self, *args):
        print('leaving context')
        #return False # -- бросаем ошибку дальше
        return True  # -- НЕ бросаем ошибку дальше

In [ ]:
try:
    with VectorWithContextManager() as vec:
        for i in range(3):
            print(i)
        raise Exception('something happened inside!')
except:
    print('an exception was raised...')
    pass
print('we are out of the context')

Но можно и проще!

In [ ]:
from contextlib import contextmanager

@contextmanager
def vector_mgr():
    print('handling entering the context')
    yield VectorWithBothReprAndStr()
    print('handling leaving the context')
          
print('statement before context')
with vector_mgr() as vector:
    for i in range(3):
        print(vector)
print('statement after context')

In [ ]:
# А теперь с ошибкой:

@contextmanager
def vector_mgr():
    try:
        print('handling entering the context')
        yield VectorWithBothReprAndStr()
    except ZeroDivisionError as e:
        pass
    finally:
        print('handling leaving the context')
          
print('statement before context')
with vector_mgr() as vector:
    for i in range(3):
        print(vector)
        infin = 1 / 0
print('statement after context')